COMP 215 - LAB 2 (NEO)
----------------
#### Name: Sos & Roberto
#### Date: Jan 16 2023

This lab exercise is mostly a review of strings, tuples, lists, dictionaries, and functions.

**Building on new concepts from lab 1**:
  * *f-string* simplifies string formatting operations (like, a lot!)
  * *list comprehension* provides a compact way to represent map and filter algorithms

**New Python Concepts**:
  * *f-string* simplifies string formatting operations (like, a lot!)

As usual, the first code cell simply imports all the modules we'll be using...

In [1]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [2]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the stringg
print(formatted_date)

2023-01-23 <class 'datetime.date'>
Today is: 2023-01-23


### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today:

In [3]:
API_KEY = 'ZA3bIx22Y3t1Rjhdcr5WiHYXPQMQILrXT8xMwnpc'  # substitute your API key here

today = '2023-01-09'   # Today's date as a string!
# Use an f-string here to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)  # recall json.loads for lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos)

12 Near Earth Objects found for 2023-01-09
[{'absolute_magnitude_h': 19.64,
  'close_approach_data': [{'close_approach_date': '2023-01-09',
                           'close_approach_date_full': '2023-Jan-09 13:59',
                           'epoch_date_close_approach': 1673272740000,
                           'miss_distance': {'astronomical': '0.0725377631',
                                             'kilometers': '10851494.854324597',
                                             'lunar': '28.2171898459',
                                             'miles': '6742806.2343975586'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '32387.3201949256',
                                                 'kilometers_per_second': '8.9964778319',
                                                 'miles_per_hour': '20124.2212622303'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 2301.5744618737,
     

Next we extract just the potentially hazerdous asteroids, using a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0):

In [4]:
hazards = [item for item in neos if item['is_potentially_hazardous_asteroid']==True]
print(f'{len(hazards)} potentially hazardous asteroids identified.')

3 potentially hazardous asteroids identified.


## Exercise 1

In the code cell below, **re-write the "list comprehension" above** as a loop so you understand how it works.
Notice how this kind of "conditional" list comprehension is a compact way to "filter" items of interest from a large data set.


In [5]:
# Your code here
hazards = []

for item in neos:
  if item['is_potentially_hazardous_asteroid']==True:
    hazards.append(item)

print(f'{len(hazards)} potentially hazardous asteroids identified.')
hazards

3 potentially hazardous asteroids identified.


[{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/2226554?api_key=ZA3bIx22Y3t1Rjhdcr5WiHYXPQMQILrXT8xMwnpc'},
  'id': '2226554',
  'neo_reference_id': '2226554',
  'name': '226554 (2003 WR21)',
  'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2226554',
  'absolute_magnitude_h': 19.64,
  'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.313729225,
    'estimated_diameter_max': 0.7015198735},
   'meters': {'estimated_diameter_min': 313.7292249562,
    'estimated_diameter_max': 701.5198735305},
   'miles': {'estimated_diameter_min': 0.1949422422,
    'estimated_diameter_max': 0.4359041053},
   'feet': {'estimated_diameter_min': 1029.2953904054,
    'estimated_diameter_max': 2301.5744618737}},
  'is_potentially_hazardous_asteroid': True,
  'close_approach_data': [{'close_approach_date': '2023-01-09',
    'close_approach_date_full': '2023-Jan-09 13:59',
    'epoch_date_close_approach': 1673272740000,
    'relative_velocity': {'kilometers_per_second': '8.996477

## Fetch Compelte Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [6]:
id = hazards[0]['id']
print(f'id: {id}')
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}/?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

id: 2226554


Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [7]:
# your code here

##### USING JOSEPH'S CODE AND INDEX & MIN FUNCTIONS TO LOCATE DESIRED TUPLE FROM APPROACH_DATES #####
def getClosest(approach_dates):
  distance = [float(date_distance[1]) for date_distance in approach_dates]
  return approach_dates[distance.index(min(distance))]


# close_approach_data = data['close_approach_data']
# approach_dates = [
#     (c['close_approach_date'], c['miss_distance']['kilometers']) 
#       for c in close_approach_data if c['orbiting_body'] == 'Earth'
# ]

# getClosest(approach_dates)
#####


##### ROBERTO'S & SOS' CODE #####
# def getClosest(closeApproachList):
#   closest = closeApproachList[0]
#   for record in closeApproachList:
#     if record['orbiting_body'] == 'Earth':
#       if float(record['miss_distance']['kilometers']) < float(closest['miss_distance']['kilometers']):
#         closest = record

#   result = (closest['close_approach_date'], closest['miss_distance']['kilometers'])
#   return result

## OPTIONAL - Take your skills to the next level...
## Exercise 3

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [8]:
# your code here
from datetime import timedelta

list_of_NEOs_per_day = []

for i in range(7):
  today = str(datetime.date.today() - timedelta(days = i))

  url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

  response = requests.request("GET", url, headers={}, data={})

  new_data = json.loads(response.text)

  list_of_NEOs_per_day.append(new_data['near_earth_objects'][today])


### Comprehensive version ###
date_distance_id = [
    (asteroid['close_approach_data'][0]['close_approach_date'], 
     asteroid['close_approach_data'][0]['miss_distance']['kilometers'], 
     asteroid['id']) 
      for each_day_data in list_of_NEOs_per_day 
        for asteroid in each_day_data
    ]

### Normal version for easier understanding ###
# for each_day_data in list_of_NEOs_per_day:
#   for asteroid in each_day_data:
#     date_distance_id.append(
#         (asteroid['close_approach_data'][0]['close_approach_date'], 
#           asteroid['close_approach_data'][0]['miss_distance']['kilometers'], 
#             asteroid['id']) #tuple carrying: close approach date, miss distance in km, and asteroid id
#         )


### Side note: erase the function to see the list of date_distance_id tuples ###
getClosest(date_distance_id)

('2023-01-20', '1993300.692164614', '3986732')